In [ ]:
import glob

datadir = '/home/dmoi/datasets/complexes/BU_all_renum'
fxfiles = glob.glob(datadir + '/*.fxout')

with open( fxfiles[100], 'r') as f:
	for l in f:
		print(l)

In [ ]:
import pandas as	pd
import tqdm
import numpy as np

header = 'Pdb	Group1	Group2	IntraclashesGroup1	IntraclashesGroup2	Interaction Energy	Backbone Hbond	Sidechain Hbond	Van der Waals	Electrostatics	Solvation Polar	Solvation Hydrophobic	Van der Waals clashes	entropy sidechain	entropy mainchain	sloop_entropy	mloop_entropy	cis_bond	torsional clash	backbone clash	helix dipole	water bridge	disulfide	electrostatic kon	partial covalent bonds	energy Ionisation	Entropy Complex	Number of Residues	Interface Residues	Interface Residues Clashing	Interface Residues VdW Clashing	Interface Residues BB Clashing'
header = header.split('\t')
def read_fxout(fxfile , header= header):
	values = False
	with open(fxfile, 'r') as f:
		try:
			for line in f:
				if values:
					ret =  dict( zip ( header , line.strip().split('\t') )	)
					return pd.DataFrame.from_dict({ ret['Pdb']: ret }  , orient='index')
				if 'Pdb	Group1	Group2	IntraclashesGroup1	IntraclashesGroup2	Interaction Energy	Backbone Hbond' in line:
					values = True
		except Exception as e:
			print(e)
			print(fxfile)

df = pd.concat([ read_fxout(fxfile) for fxfile in tqdm.tqdm(fxfiles) ] )


#change all columns to numeric if not Pdb
for col in df.columns:
	df[col] = df[col].map( lambda x : x if x != '-nan' else np.nan )
	if col != 'Pdb' and col != 'Group1' and col != 'Group2':
		df[col] = pd.to_numeric(df[col])

print(df)

In [ ]:
import matplotlib.pyplot as plt

plt.title('log abs Interaction Energy')
plt.hist(np.log( np.abs( df['Interaction Energy'][df['Interaction Energy'] != 0 ].dropna())), bins=100)
plt.show()

In [ ]:
nonzero = df[df['Interaction Energy'] != 0 ].dropna()
print(nonzero)

In [35]:
#use pdbfixer on all monomers

import pdbfixer
from openmm.app import PDBFile


outdir = '/home/dmoi/datasets/complexes/BU_all_renum_chains'

os.makedirs(outdir, exist_ok=True)

for pdb in tqdm.tqdm(nonzero.index):
	pdbfile = os.path.join(datadir, pdb.split('/')[-1] )
	if not os.path.exists(pdbfile):
		print('no pdb file for ' + pdb)
		continue
	structure = PDBParser().get_structure(pdb, pdbfile)
	for model in structure:
		for chain in model:
			try:
				outfile = os.path.join(outdir, pdb + '_' + chain.id + '.pdb')
				io = PDBIO()
				io.set_structure(chain)
				io.save(outfile)
				#fix the pdb file
				fixer = pdbfixer.PDBFixer(filename=outfile)
				fixer.findMissingResidues()
				fixer.findNonstandardResidues()
				fixer.findMissingAtoms()
				fixer.addMissingAtoms()
				fixer.addMissingHydrogens(7.0)
				fixer.removeHeterogens(False)
				fixer.findNonstandardResidues()
				fixer.replaceNonstandardResidues()

				#write the fixed pdb file	
				PDBFile.writeFile(fixer.topology, fixer.positions, open(outfile, 'w'))
			except Exception as e:
				print(e)
				print(pdb, chain.id)

  2%|█▌                                                                           | 55/2614 [09:58<6:26:33,  9.06s/it]

In [25]:
import sys 
sys.path.append('/home/dmoi/projects/foldtree2')


In [28]:
# create a dataset with shuffled chains

import  foldtree2_ecddcd as ft2
pdbfiles = glob.glob(outdir + '/*.pdb')
converter = ft2.PDB2PyG( aapropcsv = '/home/dmoi/projects/foldtree2/aaindex1.csv' )
converter.store_pyg(pdbfiles, filename='pairchains.h5', verbose = False)


  0%|                                                                                        | 0/8675 [00:00<?, ?it/s]


KeyError: 'MSE'